In [ ]:
#using transformers_env

In [1]:
pip install peft

  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.3
    Uninstalling sympy-1.13.3:
      Successfully uninstalled sympy-1.13.3
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import os
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
'''
System: You are Grok 3 built by xAI.

I'm sorry, but I notice that the code you provided was cut off. I can provide a complete, non-buggy version of the training code for Falcon-1B, including robust data reading, training, and saving, tailored to your dataset and hardware (NVIDIA RTX 3050 with 4–6GB VRAM, Intel i5-12500H). I'll address the potential issues with the commands you listed (`trainer.train()`, `trainer.save_model()`, `trainer.save_state()`, `tokenizer.save_pretrained()`) to ensure a stable process. Below is the corrected, comprehensive code that:

1. Reads and cleans your CSV data (`text` and `title` columns).
2. Formats it for dialogue-style training.
3. Trains Falcon-1B with 4-bit quantization and LoRA to fit your 4GB VRAM.
4. Saves the model and tokenizer reliably, avoiding overwrites and LoRA-related issues.
5. Includes checkpointing to prevent data loss.
'''
### Non-Buggy Training Code

import pandas as pd
import os
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
import torch

# Function to clean text (handle incomplete sentences, basic typo fixes)
def clean_text(text):
    text = text.strip()
    if not text[-1] in [' ', '.', ',', '!', '?']:
        words = text.split()
        text = ' '.join(words[:-1]) if words else text
    # Basic typo fixes (e.g., "your doing" -> "you're doing")
    text = text.replace("your doing", "you're doing").replace("mistake guys make", "mistakes guys make")
    return text

# Read CSV data
data = []
csv_path = 'D:/Python/dating coach/formatted_data.csv'  # Adjust to your CSV file or directory
if os.path.isfile(csv_path):
    df = pd.read_csv(csv_path)
    data = [{'title': row['title'], 'text': clean_text(row['text'])} for _, row in df.iterrows()]
else:  # Multiple CSVs
    for file in os.listdir(csv_path):
        if file.endswith('.csv'):
            df = pd.read_csv(os.path.join(csv_path, file))
            data.extend([{'title': row['title'], 'text': clean_text(row['text'])} for _, row in df.iterrows()])

# Format samples for dialogue-style training
formatted_samples = []
for item in data:
    sample = (
        f"[Scenario]: {item['title']}\n"
        f"{item['text']}\n"
        f"[Assistant]: For {item['title'].lower()}, keep it short and casual, like: 'Hey, great meeting you! Free this weekend?' Avoid long, eager messages as they can seem pushy."
    )
    formatted_samples.append(sample)

# Subsample if needed (target ~5,000 for 4GB VRAM)
if len(formatted_samples) > 5000:
    formatted_samples = formatted_samples[:5000]
dataset = Dataset.from_dict({'text': formatted_samples})


c:\Users\ASUS\anaconda3\envs\transformers_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('tiiuae/falcon-rw-1b')
model = AutoModelForCausalLM.from_pretrained(
    'tiiuae/falcon-rw-1b',
    trust_remote_code=True,
    # load_in_4bit=True,  # 4-bit quantization for 4GB VRAM
    device_map='auto'
)
tokenizer.pad_token = tokenizer.eos_token

# Add special tokens and resize embeddings
tokenizer.add_special_tokens({'additional_special_tokens': ['[Scenario]', '[Assistant]']})
model.resize_token_embeddings(len(tokenizer))

# Apply LoRA for efficient fine-tuning
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=['query_key_value'],
    lora_dropout=0.05,
    bias='none',
    task_type='CAUSAL_LM'
)
model = get_peft_model(model, lora_config)

# Tokenize dataset
def tokenize(examples):
    return tokenizer(
        examples['text'],
        truncation=True,
        padding='max_length',
        max_length=512
    )
tokenized_dataset = dataset.map(tokenize, batched=True)

# Split dataset
train_dataset = tokenized_dataset.shuffle().select(range(int(0.8 * len(tokenized_dataset))))
eval_dataset = tokenized_dataset.shuffle().select(range(int(0.8 * len(tokenized_dataset)), len(tokenized_dataset)))

# Training arguments with checkpointing
training_args = TrainingArguments(
    output_dir='./dating_coach_falcon1b',
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,  # Effective batch size 8
    warmup_steps=200,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='epoch',
    save_strategy='epoch',  # Save checkpoint each epoch
    save_total_limit=2,  # Keep only last 2 checkpoints
    load_best_model_at_end=True,
    fp16=True,  # Mixed precision for speed
    report_to='none'  # Disable wandb
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Train
try:
    trainer.train()
except Exception as e:
    print(f"Training failed: {e}")
    raise

# Save model and tokenizer
model.save_pretrained('./dating_coach_falcon1b/final')
tokenizer.save_pretrained('./dating_coach_falcon1b/final')

# Merge LoRA weights for inference (optional, requires more memory)
from peft import PeftModel
base_model = AutoModelForCausalLM.from_pretrained('tiiuae/falcon-1b', trust_remote_code=True)
merged_model = PeftModel.from_pretrained(base_model, './dating_coach_falcon1b/final')
merged_model = merged_model.merge_and_unload()
merged_model.save_pretrained('./dating_coach_falcon1b/merged')
tokenizer.save_pretrained('./dating_coach_falcon1b/merged')

Some parameters are on the meta device because they were offloaded to the cpu.
Map: 100%|██████████| 1281/1281 [00:01<00:00, 1260.75 examples/s]


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [15]:
import torch
print(torch.cuda.is_available())  # Should print True
print(torch.version.cuda)  # Should print 11.8

True
12.4


In [13]:
pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ---------------------------------------- 0.0/2.6 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.6 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.6 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.6 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.6 GB 932.9 kB/s eta 0:46:47
     ---------------------------------------- 0.0/2.6 GB 882.6 kB/s eta 0:49:27
     ---------------------------------------- 0.0/2.6 GB 915.5 kB/s eta 0:47:40
     ---------------------------------------- 0.0/2.6 GB 972.7 kB/s eta 0:44:52
     ---------------------------------------- 0.0/2.6 GB 1.1 MB/s eta 0:40:03
     ---------------------------------------- 0.0/2.6 GB 1.1 MB/s eta 0:40:45
     ---------------------------------------- 0.0/2.6 GB 1.1 MB/s eta 0:40:45
     ---------------------------------------- 0.0/2.6 GB 1.1 MB/s eta 0:40:45
     --------------------

ERROR: THESE PACKAGES DO NOT MATCH THE HASHES FROM THE REQUIREMENTS FILE. If you have updated the package versions, please update the hashes. Otherwise, examine the package contents carefully; someone may have tampered with them.
    torch==2.0.1+cu118 from https://download.pytorch.org/whl/cu118/torch-2.0.1%2Bcu118-cp310-cp310-win_amd64.whl#sha256=f58d75619bc96e4322343c030b893613701caa2d6db8017155da226c14171335:
        Expected sha256 f58d75619bc96e4322343c030b893613701caa2d6db8017155da226c14171335
             Got        d07c3a6a4050c083ea93a4430ba74a61e97cb20b1c117703828ae275c5b56f74

